# Problem 233: Lattice Points on a Circle

Let $f(N)$ be the number of points with integer coordinates that are on a circle passing through $(0,0)$, $(N,0)$,$(0,N)$, and $(N,N)$.
It can be shown that $f(10000) = 36$.

What is the sum of all positive integers $N \le 10^{11}$ such that $f(N) = 420$?

## Thoughts

For a given N, the problem above describes the circle

$$\frac{N}{2}^2 = \left(x - \frac{N}{2}\right)^2 + \left(y - \frac{N}{2}\right)^2$$

For any circle, centered on the origin, the number of lattice points on the circle [is given by](https://mathworld.wolfram.com/CircleLatticePoints.html) $n(R) = r_2(R^2)$ where $r_k$ is the [sum of squares function ](https://mathworld.wolfram.com/SumofSquaresFunction.html).

This means that, if $N$ is even (so that $(N/2, N/2)$ is a lattice point), $f(N) = r_2(N^2/2)$.

What if $N$ is not even? We can try to look at the circle with center at $(N, N)$ and radius $N$. This corresponds to a finer mesh and will have the same integer solutions as the circle $(N/2, N/2)$ with radius $N/2$, but may also have more integer solutions since the mesh is finer.

It turnes out that $r_2$ is invariant to input that are powers of two so $r_2(x) = r_2(2^nx)$ where $n$ is an integer $\geq 0$.

This means that, without loss of generality, we can simply consider integer solutions to circles with radius $N$ instead of $N/2$. that is $f(N) = f(2N) = r_2(2N^2)$. In fact, since factors of two are unimportant we can simply say

$f(N) = r_2(N^2)$

## Nature of $r_2(n)$

For an integer, $n$, with prime factorization $n = 2^g\prod_{i=1}p_i^{f_i}\prod_{j=1}q_j^{h_i}$ where $p_i$ and $q_i$ are prime factors that are 1 modulo 4 and 3 respectively

$$
r_2(n) = \begin{cases}
        4\prod_{i=1} f_i+1 & \text{if } h_j \text{ (all) even} \\
        0 & \text{otherwise}
    \end{cases}
$$

For $f(N)$ to be equal to $420=4\times105$ this implies that $\prod_i f_i + 1 = 105$. The prime factors of $105$ are 3, 5, and 7, so for three exponents $\{f_1+1, f_2+1, f_3+1\} = \{3, 5, 7\}$, equivalent to exponents of $N$ of $\{1, 2, 3\}$. For two exponents we can have $\{f_1+1, f_2+1\} = \{7, 15\}, \{5, 21\}, \{3, 35\}$, and for a single expontent we have $\{f_1+1\} = \{105\}$

In [46]:
from functools import cache


@cache
def generate_q_factors(qs, mx, factor=1):
    yield factor
    for q in qs:
        if factor * q <= mx:
            yield from generate_q_factors(qs, mx, factor * q)


def generate_p_factors(ps, mx):
    # three exponents (1, 2, 3)
    for p3 in ps:
        for p2 in ps:
            if p2 == p3:
                continue
            if p2 * p2 * p3 * p3 * p3 > mx:
                break
            for p1 in ps:
                if p3 == p1 or p2 == p1:
                    continue
                if p1 * p2 * p2 * p3 * p3 * p3 > mx:
                    break
                yield p1 * p2 * p2 * p3 * p3 * p3

    # two exponents (3, 7)
    for p2 in ps:
        for p1 in ps:
            if p1 == p2:
                continue
            if p2**7 * p1**3 > mx:
                break
            yield p2**7 * p1**3

    # two exponents (2, 10)
    for p2 in ps:
        for p1 in ps:
            if p1 == p2:
                continue
            if p2**10 * p1**2 > mx:
                break
            yield p2**10 * p1**2

In [47]:
from sympy import primerange

generate_q_factors.cache_clear()

nmax = 10**11

pmax = nmax // (13**2 * 5**3)
qmax = nmax // (17 * 13**2 * 5**3)

p, q = [], []

for prime in primerange(2, pmax):
    if prime % 4 == 1:
        p.append(prime)
    elif prime <= qmax:
        q.append(prime)

p_factors = sorted(generate_p_factors(p, nmax))
q_factors = sorted(generate_q_factors(tuple(q), qmax))

sumN = 0
for pf in p_factors:
    for qf in q_factors:
        if pf * qf > nmax:
            break
        sumN += pf * qf

print("Sum of positive integers such that f(N) = 420 and N <= 10^11", sumN)

Sum of positive integers such that f(N) = 420 and N <= 10^11 271204031455541309
